In [ ]:
# test script for complete get_nse_options

import pandas as pd
from ib_insync import *
util.startLoop()
from helper import get_connected, get_nse_options, get_nses

ib = get_connected('nse', 'live')

qlm = get_nses(ib)
undContracts = qlm[0]
lots_dict = qlm[1]
margins_dict = qlm[2]

tickers = ib.reqTickers(*undContracts)
undPrices = {t.contract.symbol: t.marketPrice() for t in tickers} # {symbol: undPrice}

u = undContracts[0]
p = list(undPrices.values())[0]
z = list(lots_dict.values())[0]
m = list(margins_dict.values())[0]

get_nse_options(ib, u, p, z, m)

ib.disconnect()

import pandas as pd
pd.read_pickle('../data/nse/ACC.pkl').head() 

In [ ]:
# test script for one symbol get_nse_option

from ib_insync import *
util.startLoop()
from helper import get_connected, get_nses
ib = get_connected('nse', 'live')

symbol = 'PNB'
lotsize = 7000
c = ib.qualifyContracts(Stock(symbol, 'NSE'))[0]
ib.reqSecDefOptParams(underlyingSymbol=symbol, futFopExchange='', underlyingSecType='STK', underlyingConId=c.conId)

strikes = (57.5, 60.0, 62.5)
os = [Option('PNB', '20190627', s, 'P', 'NSE') for s in strikes]
ocs = ib.qualifyContracts(*os)
order = Order(action='SELL', orderType='MKT', totalQuantity=lotsize, whatIf=True)

from helper import get_nses
qlm = get_nses(ib)

qlm[2]['PNB']

from itertools import repeat
{c.conId: ib.whatIfOrder(c, Order(action='SELL', orderType='MKT', \
                                 totalQuantity=abs(ls), whatIf=True)).initMarginChange
 for c, ls in (zip(ocs, repeat(lotsize)))}

ib.disconnect()

In [ ]:
# get_snp_remqty.py program test script
def get_snp_remqty(ib):
    '''generates the remaining quantities dictionary
    Args:
        (ib) as connection object
    Returns:
        remqty as a dictionary of {symbol: remqty}
        '''
    exchange = 'SMART'
    snp_assignment_limit = 50000
    lotsize=100
    
    # get the list of underlying contracts
    undContracts = get_snps(ib)
    c_dict = {u.symbol: u for u in undContracts} # {symbol: contract}
    
    tickers = ib.reqTickers(*undContracts)
    undPrices = {t.contract.symbol: t.marketPrice() for t in tickers} # {symbol: undPrice}

    df_und = \
        pd.DataFrame.from_dict(undPrices, orient='index', columns=['undPrice']).\
        join(pd.DataFrame.from_dict(c_dict, orient='index', columns=['undContract'])).dropna()
    
    df_und = df_und.assign(lotsize=lotsize)
    
    # remaining quantities in entire snp per assignment limit
    df_und = df_und.assign(remq=(snp_assignment_limit/(df_und.lotsize*df_und.undPrice)).astype('int')) 
    
    # from portfolio
    #_______________
    
    p = util.df(ib.portfolio()) # portfolio table
    
    # extract option contract info from portfolio table
    dfp = pd.concat([p, util.df([c for c in p.contract])[util.df([c for c in p.contract]).columns[:7]]], axis=1).iloc[:, 1:]
    dfp = dfp.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

    # get the underlying's margins, lots, prices and positions
    pos_dict = dfp.groupby('symbol')['position'].sum().to_dict()
    p_syms = {s for s in dfp.symbol}
    p_undc = {s: ib.qualifyContracts(Stock(s, exchange, currency='USD')) for s in p_syms}   # {symbol: contracts} dictionary
    undmlist = [get_margins(ib, u, 100) for u in p_undc.values()]                 # {contract: margin} dictionary
    p_undMargins = {k.symbol: v for i in undmlist for k, v in i.items()}        # {symbol: margin} dictionary
    p_undPrices = {u: undPrices[u] for u in p_syms}    #{symbol: undPrice} dictionary
    
    dfp1 = pd.DataFrame.from_dict(p_undc, orient='index', columns=['contract']). \
        join(pd.DataFrame.from_dict(p_undMargins, orient='index', columns=['undmargin'])). \
        join(pd.DataFrame.from_dict(p_undPrices, orient='index', columns=['undPrice'])). \
        join(pd.DataFrame.from_dict(pos_dict, orient='index', columns=['position']))
    
    dfp1 = dfp1.assign(undLot=1)
    
    dfp1 = dfp1.assign(qty=(dfp1.position/dfp1.undLot).astype('int'))
    
    # make the blacklist
    #___________________
    remqty_dict = pd.DataFrame(df_und.loc[dfp1.index].remq + dfp1.qty).to_dict()[0] # remq from portfolio
    remqty_dict = {k:(v if v > 0 else 0) for k, v in remqty_dict.items()} # portfolio's remq with negative values removed
    blacklist = [k for k, v in remqty_dict.items() if v <=0] # the blacklist
    df_und.remq.update(pd.Series(remqty_dict)) # replace all underlying with remq of portfolio
    remqty = df_und.remq.to_dict() # dictionary
    return remqty

In [ ]:
import pandas as pd
from ib_insync import *
util.startLoop()
from helper import get_connected, get_snps, get_margins
# ib = get_connected('snp', 'live')
remqty = get_snp_remqty(ib)
ib.disconnect()

In [ ]:
import pandas as pd
import numpy as np

#***      Function to print python code in Jupyter   ****
#________________________________________________________

def display_py(code):
    """Displays python file code in Jupyter
    Arg: (string from py file) code
    Output: code formatted for jupyter
    Usage:
    with open(myfile) as f:
         code = f.read()
    display_py(code)
    """
    from pygments import highlight
    from pygments.lexers import PythonLexer
    from pygments.formatters import HtmlFormatter
    import IPython
    
    formatter = HtmlFormatter()

    html_code = highlight(code, PythonLexer(), HtmlFormatter())
    styled_html = '<style type="text/css">{}</style>{}'.format(formatter.get_style_defs('.highlight'), html_code)
    ipython_code = IPython.display.HTML(styled_html)

    return ipython_code

In [ ]:
from helper import get_connected
??get_connected

In [14]:
from os import listdir
import pandas as pd

from helper import get_connected, grp_opts, get_prec, recalc_opts

fspath = '../data/nse/'  # path for nse pickles

minOptPrice = 0.15
minRom = 0.1

nLargest = 5

fs = listdir(fspath)

cols = ['optId', 'symbol', 'right', 'expiration', 'dte', 'strike', 'undPrice', 
'lo52', 'hi52', 'Fall', 'Rise', 'loFall', 'hiRise', 'std3', 'loStd3', 'hiStd3', 
'lotsize', 'optPrice', 'optMargin', 'rom']

optsList = [f for f in fs if f[-3:] == 'pkl']

df1 = pd.concat([pd.read_pickle(fspath+f) 
                 for f in optsList], axis=0, sort=True).reset_index(drop=True)[cols]

In [23]:
tp = pd.read_html('https://www.tradeplusonline.com/Equity-Futures-Margin-Calculator.aspx')

df_tp = tp[1][2:].iloc[:, :-1]
df_tp = df_tp.iloc[:, [0,1,5]]
df_tp.columns=['nseSymbol', 'lot', 'margin']

cols = df_tp.columns.drop('nseSymbol')
df_tp[cols] = df_tp[cols].apply(pd.to_numeric, errors='coerce') # convert lot and margin to numeric

df_slm = df_tp.copy()

# Truncate to 9 characters for ibSymbol
df_slm['ibSymbol'] = df_slm.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_slm.ibSymbol = df_slm.ibSymbol.replace(ntoi)

In [35]:
pickled_lotsize = df1.drop_duplicates('symbol')[['symbol', 'lotsize']].reset_index(drop=True)

In [36]:
raw_lotsize = df_slm[['ibSymbol', 'lot']].reset_index(drop=True)
raw_lotsize.columns=list(pickled_lotsize)

In [42]:
pdict = pickled_lotsize.set_index('symbol').to_dict()['lotsize']
rdict = raw_lotsize.set_index('symbol').to_dict()['lotsize']

In [77]:
{k:v for k,v in pdict.items() if k not in rdict or v != rdict[k]}

{'BANKNIFTY': 900,
 'UNIONBANK': 1000,
 'UPL': 700,
 'VEDL': 1600,
 'VGUARD': 200,
 'VOLTAS': 7000,
 'WIPRO': 1200,
 'WOCKPHARM': 2300,
 'YESBANK': 3000,
 'ZEEL': 1000}

In [62]:
{k: rdict[k] for k in set(rdict) - set(pdict)}

{'IDEA': 19868, 'NIFTY50': 75, 'NIFTYIT': 50, 'LUPIN': 700}

In [76]:
rdict.items() | pdict.items()

{('ACC', 400),
 ('ADANIENT', 4000),
 ('ADANIPORT', 2500),
 ('ADANIPOWE', 20000),
 ('AJANTPHAR', 500),
 ('ALBK', 13000),
 ('AMARAJABA', 700),
 ('AMBUJACEM', 2500),
 ('APOLLOHOS', 500),
 ('APOLLOTYR', 3000),
 ('ARVIND', 2000),
 ('ASHOKLEY', 4000),
 ('ASIANPAIN', 600),
 ('AUROPHARM', 1000),
 ('AXISBANK', 1200),
 ('BAJAJ-AUT', 250),
 ('BAJAJFINS', 125),
 ('BAJFINANC', 250),
 ('BALKRISIN', 800),
 ('BANKBAROD', 4000),
 ('BANKINDIA', 6000),
 ('BANKNIFTY', 20),
 ('BANKNIFTY', 900),
 ('BATAINDIA', 550),
 ('BEL', 6000),
 ('BEML', 700),
 ('BERGEPAIN', 2200),
 ('BHARATFIN', 500),
 ('BHARATFOR', 1200),
 ('BHARTIART', 1700),
 ('BHEL', 7500),
 ('BIOCON', 900),
 ('BOSCHLTD', 30),
 ('BPCL', 1800),
 ('BRITANNIA', 200),
 ('BSOFT', 2250),
 ('CADILAHC', 1600),
 ('CANBK', 2000),
 ('CANFINHOM', 1800),
 ('CASTROLIN', 3400),
 ('CEATLTD', 400),
 ('CENTURYTE', 600),
 ('CESC', 550),
 ('CGPOWER', 12000),
 ('CHENNPETR', 1800),
 ('CHOLAFIN', 500),
 ('CIPLA', 1000),
 ('COALINDIA', 2200),
 ('COLPAL', 700),
 ('CONCOR',

In [69]:
df1[df1.symbol == 'VOLTAS'].head()

,optId,symbol,right,expiration,dte,strike,undPrice,lo52,hi52,Fall,Rise,loFall,hiRise,std3,loStd3,hiStd3,lotsize,optPrice,optMargin,rom
5209,359500793,VOLTAS,C,20190425,7,650.0,623.85,472.25,675.3,89.9,86.15,533.95,710.0,22.289628,601.560372,646.139628,7000,4.45,120328,9.319527
5210,359500756,VOLTAS,P,20190425,7,600.0,623.85,472.25,675.3,89.9,86.15,533.95,710.0,22.289628,601.560372,646.139628,7000,4.40,120328,9.214813
5211,359500801,VOLTAS,C,20190425,7,660.0,623.85,472.25,675.3,89.9,86.15,533.95,710.0,22.289628,601.560372,646.139628,7000,2.65,120328,5.549830
5212,359500751,VOLTAS,P,20190425,7,590.0,623.85,472.25,675.3,89.9,86.15,533.95,710.0,22.289628,601.560372,646.139628,7000,2.50,120328,5.235689
5213,359500811,VOLTAS,C,20190425,7,670.0,623.85,472.25,675.3,89.9,86.15,533.95,710.0,22.289628,601.560372,646.139628,7000,1.55,120328,3.246127


In [73]:
rdict['VOLTAS']

1000

In [72]:
df_slm[df_slm.ibSymbol == 'VOLTAS']

,nseSymbol,lot,margin,ibSymbol
196,VOLTAS,1000,108978,VOLTAS


In [ ]:
# check an existing pickle
import pandas as pd
pd.read_pickle('../data/nse/VOLTAS.pkl')

In [8]:
from ib_insync import *
util.startLoop()
from helper import get_nses, get_connected

exchange = 'NSE'
nse_assignment_limit = 1500000

# get the list of underlying contracts and dictionary of lots
with get_connected('nse', 'live') as ib:
    qlm = get_nses(ib)
    c_dict = qlm[0]       # {symbol: contract}
    lots_dict = qlm[1]    # {symbol: lotsize}
    margin_dict = qlm[2]  # {symbol: margin}
    undContracts = [v for k, v in c_dict.items()]

Started to throttle requests
Stopped to throttle requests


In [10]:
c_dict['BANKNIFTY']

Index(conId=56994300, symbol='BANKNIFTY', exchange='NSE', currency='INR', localSymbol='BANKNIFTY')